# 개요

- 타겟 사이트의 데이터까지 접근하는 난이도 높음
  - 사용자 인터렉션
  - 보안 이슈
  - 기타 작업 병행 -> 사용자의 노력 필요
    - 로그인, 스크롤, 검색, 클릭, 대기, 자바스크립트,...
  - 웹 브라우저상 화면이 변화
    - form 전공을 통한 화면 갱신 (DOM 모든 주소 새로 갱신) -> 껌뻑 -> 처음부터 다시
    - ajax 전송 -> DOM주소들 그대로 유지됨 -> 기존정보 그대로 사용
    - 유튜브 더보기, 댓글 자동 불러오기, 검색

  - 데이터 수집 참고 사이트
    - opinet
    - youtube
    - starbucks

- 준비
  - 라이브러리
    - selenium
  - web driver
    - 프로그램에서 제어 -> 브라우저 조작
      - 사이트 접속. 아이디비번 입력, 로그인, 검색, 스크롤, 글작성... => 사이트 테스트 <-> 매크로
    - 브라우저 밴더들이 제공
      - 크롬, 사파리, 엣지, 파폭,...
    - 타겟 브라우저 크롬
      - 현재 사용중인 **크롬의 버전과 일치하는 드라이버 획득해야함**
      - https://googlechromelabs.github.io/chrome-for-testing/
        - 현재 stable > 135 버전 진입 > chrome webdriver win64 버전 다운 > 압축 해제 > *.exe => projects/crawling 하위에 위치

In [26]:
# 별도 설치 필요함
!pip install selenium -q


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [27]:
# 모듈확인
from selenium import webdriver as wd

In [28]:
# 웹드라이버용으로 다운받은 브라우저 가동 -> 물리적 가동
driver = wd.Chrome()

In [29]:
# 3. 접속 -> 타겟사이트, GET방식
TARGET_SITE = 'https://www.opinet.co.kr/user/main/mainView.do'

driver.get(TARGET_SITE)

# 데이터 수집하는 환경(인터넷 속도가 다름)
# 보수적으로 명시적 딜레이 부여 (명시적, 암묵적)
import time
time.sleep(2) # 2초간 강제로 대기 -> 요청한 화면이 완료될때까지

In [57]:
# 첫번째 수집 정보 => 시도 정보 추출 : <option>의 value 속성값 추출해라
# 분석 -> css selector 도출
# 1. 시도 요소(select 태그) css selector : # SIDO_NMO
# 2. <select>..<option></select> 이 구조상의 option의 value 추출
# -> 사용자가 선택시 적용되는 실제값임
# css selector : #SIDO_NMO > option

# 1. 시도 select 요소를 특정 -> 1개만 존재
# driver.find_element(특정방법 특정문자열)
from selenium.webdriver.common.by import By

sido = driver.find_element(By.CSS_SELECTOR, '#SIDO_NM0')

# 2. 시도 select 요소 하위(자식)에 존재하는 option 요소 모두(s) 특정
sido_opts = driver.find_elements(By.CSS_SELECTOR, '#SIDO_NM0 > option')

len(sido_opts)

18

In [58]:

# 데이터 추출
for sido_opt in sido_opts[1:]:
    print(sido_opt.get_attribute('value'))
    #break

서울특별시
부산광역시
대구광역시
인천광역시
광주광역시
대전광역시
울산광역시
세종특별자치시
경기도
강원특별자치도
충청북도
충청남도
전북특별자치도
전라남도
경상북도
경상남도
제주특별자치도


In [59]:
# sido value
sido_values = [
    sido_opt.get_attribute('value')
    for sido_opt in sido_opts[1:]

]
sido_values

['서울특별시',
 '부산광역시',
 '대구광역시',
 '인천광역시',
 '광주광역시',
 '대전광역시',
 '울산광역시',
 '세종특별자치시',
 '경기도',
 '강원특별자치도',
 '충청북도',
 '충청남도',
 '전북특별자치도',
 '전라남도',
 '경상북도',
 '경상남도',
 '제주특별자치도']

In [60]:
# 사용자가 시도 다른 지역을 선택 -> 화면 변화함(해당 지역으로 정보 변경됨)
# 선택 => 클릭(버튼클릭x, 서울선택, 부산선택,...) => 데이터를<select> 입력한 행위임(여기서는)
# 데이터 입력처리 함수 => send_keys(값)
import time

# 전국 시도를 순환하면서 선택 테스트 => 각 시도에는 5초정도 머물고 이동하겠음
for sido_values in sido_values:
    print(sido_values)
    # 시도 select 요소를 매번 찾아서 관리, 향후 화면이 새로고침되는 상황이 발생할때 문제x
    sido = driver.find_element(By.CSS_SELECTOR,'#SIDO_NM0')
    # select 요소에 입력
    sido.send_keys(sido_values) # 서울->부산->...->제주도
    # 잠시대기 -> 5초
    time.sleep(5)

서울특별시


KeyboardInterrupt: 

In [63]:
# 편의상 서울만 진행
for sido_value in sido_values:
    print( sido_values )

    # 시도 선택 완료
    sido = driver.find_element( By.CSS_SELECTOR, '#SIDO_NM0' )
    sido.send_keys( sido_values ) # 서울 선택
    time.sleep( 5 )

    # 시군구
    # 시군구 <select> 요소 특정
    # sigungu = driver.find_element(By.CSS_SELECTOR, '#SIGUNGU_NM0')
    # 시군구 <select> 요소의 자식 <ootion> 모두 찾기
    sigungu_opts = [
        # sigungu => <option value = "강남구">강남구</option>
        sigungu.get_attribute('value') # 강남구, 강서구,....

        # 시군구 select 요소 하위에 존재하는 모든 option 요소를 찾아서 -> 반복문 처리
        for sigungu in driver.find_elements(By.CSS_SELECTOR, '#SIGUNGU_NM0 > option')
    ]
    print(sigungu_opts)

    # 강남구 선택 -> 잠시대기 -> 엑셀파일 다운로드 -> 다음.. -> 중랑구 집행완료 -> 25개의 엑셀 수집됨
    for sigungu_opt in sigungu_opts[1:]: # 실제 자치구 데에터만 반복
        # 1. 시군구 select 요소 찾음
        sigungu = driver.find_element(By.CSS_SELECTOR, '#SIGUNGU_NM0')
        # 2. 현재 시군구 값 세팅
        sigungu.send_keys(sigungu_opt) # 강남구, 강서구, ..중랑구
        # 3. 잠시 대기 (엑셀 만들어지고, 화면 세팅되는 시간 대기) -> 명시적 대기
        time.sleep(4) # 난수를 개입 => 변조가 가능함
        # 4. 엑셀 다운 저장 버튼 클릭
        driver.find_element(By.CSS_SELECTOR,'.btn_type6_ex_save').click()
        # 5. 엑셀이 브라우저에 다운될때까지 대기 -> 명시적
        time.sleep(3)

    # 서울만 처리 -> 1회 반복후 종료
    break

서울특별시
['', '강남구', '강동구', '강북구', '강서구', '관악구', '광진구', '구로구', '금천구', '노원구', '도봉구', '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구', '양천구', '영등포구', '용산구', '은평구', '종로구', '중구', '중랑구']


In [64]:
# 브라우저 종료
driver.close()
driver.quit()

MaxRetryError: HTTPConnectionPool(host='localhost', port=59376): Max retries exceeded with url: /session/eca9493e7798d4887d860cc2477888d7/window (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001D2D772F8A0>: Failed to establish a new connection: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다'))

KeyboardInterrupt: 

# 데이터 적제

- xls -> DataFrame

In [65]:
# 1. 다운로드된 모든 xls의 목록을 가져온다 -> glob

import glob

ERROR! Session/line number was not unique in database. History logging moved to new session 4


In [ ]:
# 특정 폴더에 존재하는 엑셀파일을 모두 가져온다
files = glob.glob('C://Users//user//Downloads//지역_위치별(주유소)*.xls')
len(files)

25

In [ ]:
엑셀 파일을 처리하기 위한 라이브러리 설치치
!pip install -q fsspec xlrd


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [73]:
# 파일을 하나씩 읽어서 DataFrame으로 구성
# [df, df, df, df, df,....]
import pandas as pd

dfs = list()
for file in files:
    # 상위 2줄은 데이터가 x, 3번째 줄부터 컬럼(헤더)에 해당, 나머진 데이터임
    df = pd.read_excel(file,header=2) # xls -> DataFrame
    dfs.append (df)

len(dfs)

25

In [ ]:
# 마무리 작업, 25개의 df를 1개의 df러 병합
oil_seoul_price_df = pd.concat(dfs)

# 데이터 전체 볼륨
oil_seoul_price_df.shape
# 서울시 주유소의 총개수 422개

(422, 10)

In [ ]:
# 샘플확인 (무작위로 10개)
oil_seoul_price_df.sample(10)

,지역,상호,주소,상표,전화번호,셀프여부,고급휘발유,휘발유,경유,실내등유
21,서울특별시,국회대로주유소,서울 영등포구 국회대로 746 (여의도동),HD현대오일뱅크,02-761-5122,Y,2198,1769,1659,-
6,서울특별시,동자동주유소,서울 용산구 한강대로104길 6 (동자동),SK에너지,02-754-6667,N,2399,2291,2048,1650
11,서울특별시,SK에너지㈜직영 상계주유소,서울 노원구 노해로 527 (상계동),SK에너지,02-930-5151,N,-,1729,1599,-
3,서울특별시,지에스칼텍스㈜ 동서울주유소,서울 강동구 천호대로 1456 (상일동),GS칼텍스,02-426-5372,Y,1919,1699,1599,-
11,서울특별시,송파스마일주유소,서울 송파구 송이로 28,GS칼텍스,02-2203-3737,Y,1853,1653,1533,-
9,서울특별시,범아주유소,서울 중랑구 동일로 881,S-OIL,02-974-8356,N,-,1695,1539,1600
0,서울특별시,만남의광장주유소,서울 서초구 양재대로12길 73-71,알뜰(ex),02-573-7430,Y,-,1619,1479,-
10,서울특별시,용마로주유소,서울 중랑구 용마산로 309 (면목동),SK에너지,02-439-3037,Y,-,1698,1598,1600
20,서울특별시,제트라인DM주유소,서울 송파구 중대로 183 (가락동),HD현대오일뱅크,02-430-5151,N,-,1685,1578,-
0,서울특별시,주식회사 새서울글로벌,서울 동작구 시흥대로 616 (신대방동),HD현대오일뱅크,02-832-7926,Y,1875,1599,1488,-
